In [1]:
# STEP 1: Install Required Libraries
# Gradio provides the GUI interface, pandas handles data processing
!pip install -q gradio pandas

In [2]:
# STEP 2: Import Dependencies
import pandas as pd
import gradio as gr
import numpy as np

In [3]:
# STEP 3: Define Nutrition Database
# Complete nutritional data per standard unit (100g, 1 tbsp, 1 unit, etc.)
# Each ingredient contains 15 nutritional values in the following order:
# [Calories, Carbs, Fat, Protein, Sat.Fat, Trans Fat, Cholesterol, Sodium, Potassium, 
#  Fiber, Sugar, Vit.A, Vit.C, Calcium, Iron]

NUTRITION_DATA = {
    'Raw Rice (x 100g)':      [360, 79.5, 0.6, 7.0, 0.1, 0.0, 0, 1, 110, 1.0, 0.1, 0, 0, 0, 1],
    'Meat (x 100g)':          [250, 0.0, 17.0, 25.0, 6.5, 0.0, 75, 75, 300, 0.0, 0.0, 0, 0, 0, 10],
    'Chk/Turky (x 100g)':     [165, 0.0, 3.6, 31.0, 1.0, 0.0, 85, 70, 450, 0.0, 0.0, 0, 0, 0, 4],
    'Salmon (x 100g)':        [208, 0.0, 13.0, 22.0, 3.0, 0.0, 60, 60, 490, 0.0, 0.0, 2, 0, 0, 5],
    'Shr (x 100g)':           [130, 0.0, 2.5, 25.0, 0.5, 0.0, 200, 100, 200, 0.0, 0.0, 0, 0, 1, 10],
    'Oil (x 1 tbsp)':         [120, 0.0, 13.5, 0.0, 2.0, 0.0, 0, 0, 1, 0.0, 0.0, 0, 0, 0, 0],
    'Split Peas (x 100g)':    [360, 60.3, 1.0, 24.5, 0.2, 0.0, 0, 15, 960, 25.0, 5.0, 0, 0, 1, 40],
    'Lentils (x 100g)':       [350, 60.1, 1.0, 25.0, 0.2, 0.0, 0, 5, 900, 30.0, 2.0, 0, 0, 1, 37],
    'Beans/Chic (x 100g)':    [350, 63.0, 6.0, 20.0, 0.6, 0.0, 0, 20, 870, 17.0, 11.0, 0, 0, 1, 37],
    'Egg (x 1 unit)':         [70, 0.6, 5.0, 6.0, 1.5, 0.0, 185, 70, 70, 0.0, 0.3, 4, 0, 2, 5],
    'Onion (x 100g)':         [40, 9.3, 0.1, 1.1, 0.0, 0.0, 0, 4, 145, 1.7, 4.2, 0, 10, 2, 1],
    'Paste/Tomato (x 1 tbsp)':[20, 4.8, 0.1, 1.0, 0.0, 0.0, 0, 60, 160, 1.0, 3.0, 5, 10, 1, 4],
    'Herbs/Veg (x 10g)':      [30, 6.0, 0.5, 3.0, 0.1, 0.0, 0, 100, 150, 3.0, 1.0, 20, 10, 5, 15],
    'Macaroni (x 100g)':      [370, 74.0, 1.5, 13.0, 0.5, 0.0, 0, 5, 150, 3.0, 1.0, 0, 0, 1, 8],
    'Cheese (x 100g)':        [350, 3.1, 29.0, 25.0, 18.0, 0.0, 100, 620, 90, 0.0, 0.5, 10, 0, 72, 1],
    'Nuts (x 100g)':          [650, 21.0, 56.0, 22.0, 4.5, 0.0, 0, 1, 700, 12.0, 4.0, 0, 0, 7, 15],
    'Potato (x 100g)':        [75, 17.5, 0.1, 2.0, 0.0, 0.0, 0, 6, 420, 2.0, 0.8, 0, 30, 0, 3],
    'Flour/Sugar (x 100g)':   [400, 83.0, 1.0, 10.0, 0.3, 0.0, 0, 1, 100, 3.0, 0.2, 0, 0, 1, 10],
    'sausage (x 100g)':       [300, 1.0, 28.0, 12.0, 10.0, 0.0, 60, 800, 200, 0.0, 1.0, 0, 0, 1, 5],
}

# Define nutrient names and their units for output display
NUTRIENTS = {
    'Total Calorie': '(kCal)',
    'Total carbohydrate': '(g)',
    'Total Fat': '(g)',
    'Protein': '(g)',
    'Saturated Fat': '(g)',
    'Trans Fat': '(g)',
    'Cholesterol': '(mg)',
    'Sodium': '(mg)',
    'Potassium': '(mg)',
    'Dietary Fiber': '(g)',
    'Sugar': '(g)',
    'Vitamin A': '(% DV)',
    'Vitamin C': '(% DV)',
    'Calcium': '(% DV)',
    'Iron': '(% DV)',
}
NUTRIENT_NAMES = list(NUTRIENTS.keys())

print(f"✓ Loaded {len(NUTRITION_DATA)} ingredients")
print(f"✓ Tracking {len(NUTRIENTS)} nutritional values")

✓ Loaded 19 ingredients
✓ Tracking 15 nutritional values


In [4]:
# STEP 4: Define Calculation Logic
def calculate_nutrition(*quantities):
    """
    Calculates the total nutritional content of a meal given the quantity of each ingredient.
    
    Args:
        *quantities: Variable number of quantity values (one per ingredient)
    
    Returns:
        pandas.DataFrame: A table showing nutrient amounts and units
    """
    # Convert the tuple of quantities to a NumPy array for vectorized multiplication
    quantities = np.array(quantities)
    
    # Convert NUTRITION_DATA dictionary to a DataFrame for easy calculation
    df_nutrition = pd.DataFrame.from_dict(NUTRITION_DATA, orient='index', columns=NUTRIENT_NAMES)
    
    # Calculate total nutrients (Quantity * Nutrient_Value_Per_Unit)
    # The result is a Pandas Series where the index is the nutrient name and the value is the total amount.
    total_nutrients = df_nutrition.T.dot(quantities).round(1)
    
    # Format the output into a clean DataFrame for display
    df_output = pd.DataFrame(total_nutrients, columns=['Amount'])
    df_output['Unit'] = list(NUTRIENTS.values())
    df_output = df_output[['Amount', 'Unit']]  # Reorder columns
    df_output.index.name = 'Nutrient'
    
    # Reset any negative values (e.g., from an empty input) to 0
    df_output.loc[df_output['Amount'] < 0, 'Amount'] = 0.0
    
    return df_output

print("✓ Calculation function ready")

✓ Calculation function ready


In [5]:
# STEP 5: Create Gradio Interface
# Build input components - one number input for each ingredient
ingredient_inputs = []
for ingredient_name in NUTRITION_DATA.keys():
    # Extract the short name and unit from the full ingredient name
    short_name = ingredient_name.split(' ')[0]
    unit = ingredient_name.split('(')[1].rstrip(')')
    
    ingredient_inputs.append(gr.Number(
        label=f"Quantity of {short_name} ({unit}):",
        value=0,
        precision=1  # Allows for fractional units like 0.5 tbsp
    ))

# Create the Gradio interface
iface = gr.Interface(
    fn=calculate_nutrition,
    inputs=ingredient_inputs,
    outputs=gr.DataFrame(
        headers=["Amount", "Unit"],
        col_count=(2, 'fixed'),
        row_count=(len(NUTRIENTS), 'fixed'),
        type="pandas"
    ),
    title="🍽️ Meal Nutrition Calculator",
    description="Enter the quantity of each ingredient used in your meal to calculate the total nutritional value. Use decimals for partial units (e.g., 0.5 for half a tablespoon).",
    allow_flagging='never',
    examples=[
        # Example 1: Simple rice and chicken meal
        [1.5, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0.5, 0, 1, 0, 0, 0, 0, 0, 0],
        # Example 2: Pasta with cheese
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0.5, 1, 0, 2, 0.5, 0, 0, 0, 0],
    ]
)

print("✓ Gradio interface configured")

✓ Gradio interface configured


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


In [6]:
# STEP 6: Launch the Application
# This will create a web interface accessible via a public link
print("🚀 Launching Nutrition Calculator GUI...")
print("📱 Click the link below to open the interface")
print("💡 Tip: You can share the public link with others!\n")

iface.launch(share=True, inbrowser=True)

🚀 Launching Nutrition Calculator GUI...
📱 Click the link below to open the interface
💡 Tip: You can share the public link with others!

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b14b0b01fb09a4200e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://b14b0b01fb09a4200e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
